# Project

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import tensorboard
from torchvision import datasets
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score, confusion_matrix, plot_confusion_matrix, f1_score, precision_recall_curve

import os
import matplotlib.pyplot as plt

from textwrap import wrap
import re
import itertools
import tfplot
import matplotlib
import numpy as np
from sklearn.metrics import confusion_matrix

import time
import copy
import numpy as np
import pandas as pd
import seaborn as sns
import datetime

In [2]:
model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet50', pretrained=True)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet34', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet152', pretrained=True)
model.fc = nn.Linear(2048,2)
model.eval()


Using cache found in C:\Users\andre/.cache\torch\hub\pytorch_vision_v0.9.0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
def create_confused_matrix(labels, predicted):
    conf_mat = confusion_matrix(labels, predicted)
    hetkarta = sns.heatmap(conf_mat, annot=True)
    hetkarta = hetkarta.get_figure()
    writer.add_figure(model_name + ' heatMappad' + str(datetime.datetime.now()), hetkarta)


In [3]:
transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize(256),
    torchvision.transforms.CenterCrop(224),
    #torchvision.transforms.Grayscale(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

In [4]:
batch_size = 1000
batch_size_training = 8

dataset_train = datasets.ImageFolder('Dataset/train', 
    transform=transforms)
dataset_test = datasets.ImageFolder('Dataset/test',
    transform = transforms)
dataset_val = datasets.ImageFolder('Dataset/val',
    transform = transforms)

train_loader = DataLoader(dataset_train, batch_size=batch_size_training, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)

In [19]:
# print(next(iter(test_loader))[0])

tensor([[[[ 0.4166,  0.3652,  0.3823,  ...,  1.1700,  1.2214,  1.2214],
          [ 0.4166,  0.4166,  0.4679,  ...,  1.2043,  1.2043,  1.2214],
          [ 0.3994,  0.3994,  0.4508,  ...,  1.1872,  1.1872,  1.2043],
          ...,
          [-0.3027, -0.1143,  0.0227,  ...,  0.6049,  0.5707,  0.6049],
          [-0.0801,  0.0398,  0.1254,  ...,  0.6392,  0.6049,  0.6392],
          [ 0.0398,  0.1597,  0.1939,  ...,  0.6906,  0.6734,  0.6563]],

         [[ 0.5553,  0.5028,  0.5203,  ...,  1.3256,  1.3782,  1.3782],
          [ 0.5553,  0.5553,  0.6078,  ...,  1.3606,  1.3606,  1.3782],
          [ 0.5378,  0.5378,  0.5903,  ...,  1.3431,  1.3431,  1.3606],
          ...,
          [-0.1800,  0.0126,  0.1527,  ...,  0.7479,  0.7129,  0.7479],
          [ 0.0476,  0.1702,  0.2577,  ...,  0.7829,  0.7479,  0.7829],
          [ 0.1702,  0.2927,  0.3277,  ...,  0.8354,  0.8179,  0.8004]],

         [[ 0.7751,  0.7228,  0.7402,  ...,  1.5420,  1.5942,  1.5942],
          [ 0.7751,  0.7751,  

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    model.to('cuda')
    
model_name = "model_pretrained_10epochs"
writer = SummaryWriter("runs/" + model_name)

In [12]:
# model_name = "model_pretrained_10epochs"
# writer = SummaryWriter("runs/" + model_name)

In [6]:
torch.cuda.empty_cache()
auroc_scores = []
losses = []
losses_val = []
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    total = 0
    correct = 0
    total_val = 0
    correct_val = 0
    print("Training...")
    for i, data in enumerate(train_loader, 0):
        print( i, end="\r")
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        #inputs, labels = data
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs).to(device)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # print statistics
        running_loss += loss.item()
        writer.add_scalar('Loss/train', loss.item(), i)
        writer.add_scalar('Accuracy/train', correct/total, i)
       
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
            print("Accuracy: ", correct/total)
    print("Training Accuracy: ", correct/total)
            
    print("Validating...")
    for i, data in enumerate(val_loader, 0):
        images, labels = data[0].to(device), data[1].to(device)
        #images, labels = data
        outputs = model(images).to(device)
        loss = criterion(outputs, labels)
        writer.add_scalar('Loss/validation', loss.item(), i)
        writer.add_scalar('Accuracy/validation', correct/total, i)
        _, predicted = torch.max(outputs.data, 1)
        
        try:
            auroc = roc_auc_score(labels.to("cpu"), predicted.to("cpu"))
            auroc_scores.append(auroc)
        except ValueError:
            print("ValueError!!")
            pass
        
        total_val += labels.size(0)
        correct_val += (predicted == labels).sum().item()
    print("Validation Accuracy: ", correct_val/total_val)

print('Finished Training')

Training...
Training Accuracy:  0.9459355828220859
Validating...
Validation Accuracy:  0.8125
Training...
Training Accuracy:  0.973159509202454
Validating...
Validation Accuracy:  0.6875
Training...
Training Accuracy:  0.9789110429447853
Validating...
Validation Accuracy:  0.9375
Training...
Training Accuracy:  0.9867714723926381
Validating...
Validation Accuracy:  1.0
Training...
Training Accuracy:  0.9884969325153374
Validating...
Validation Accuracy:  0.625
Training...
Training Accuracy:  0.9913726993865031
Validating...
Validation Accuracy:  0.6875
Training...
Training Accuracy:  0.9950153374233128
Validating...
Validation Accuracy:  0.9375
Training...
Training Accuracy:  0.9973159509202454
Validating...
Validation Accuracy:  1.0
Training...
Training Accuracy:  0.9988496932515337
Validating...
Validation Accuracy:  0.5625
Training...
Training Accuracy:  0.9978911042944786
Validating...
Validation Accuracy:  0.6875
Finished Training


In [7]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# if torch.cuda.is_available():
device = "cpu"
model.to('cpu')

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [12]:
correct = 0
total = 0
total_auroc = 0
labels_total = []
auroc_scores = []

# print(type(test_loader))

with torch.no_grad():
    for i ,data in enumerate(test_loader, 0):
        print(i, end="\r")
#         images, labels = data[0].to(device), data[1].to(device)
        images, labels = data[0], data[1]
        #images, labels = data
        outputs = model(images)
        print(outputs.data)
        _, predicted = torch.max(outputs.data, 1)

        try:
            auroc = roc_auc_score(labels.to("cpu"), predicted.to("cpu"))
            auroc_scores.append(auroc)
        except ValueError:
            pass
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        create_confused_matrix(labels.to("cpu"), predicted.to("cpu"))
        f1 = f1_score(labels, predicted)
        precision_recall_curve(labels, outputs.data)

        
print('Accuracy of the network on the', total ,'test images: %d %%' % (
    100 * correct / total))
print("Auoroc: ", np.mean(auroc_scores))
print("F1: ", f1)

tensor([[-10.9728,  11.9634],
        [ -7.9361,   8.1671],
        [  0.3572,  -1.0122],
        ...,
        [-11.1826,  11.3626],
        [-12.4472,  12.8989],
        [-13.5672,  13.7980]])


ValueError: y should be a 1d array, got an array of shape (624, 2) instead.

In [14]:
print(torch.softmax(torch.max(outputs.data)))

TypeError: softmax() received an invalid combination of arguments - got (Tensor), but expected one of:
 * (Tensor input, name dim, *, torch.dtype dtype)
 * (Tensor input, int dim, torch.dtype dtype)


In [ ]:
outputs.data.shape

# Load and Save Model

In [ ]:
model.load_state_dict(torch.load(model_name + ".pt"))
model.to(device)

In [ ]:
torch.save(model.state_dict(), model_name +".pt")

# TODO:
## precision recall
## more epochs (300 epochs)